In [3]:
!pip install -q -U  langchain_openai langchain langgraph rich python-dotenv tavily-python

zsh:1: command not found: pip


In [ ]:
!pip install 


zsh:1: command not found: pip


In [17]:
pip install --upgrade rich #langchain_community #langchain_openai #langchain-tavily


  Using cached rich-14.2.0-py3-none-any.whl.metadata (18 kB)
  Using cached markdown_it_py-4.0.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
Using cached rich-14.2.0-py3-none-any.whl (243 kB)
Using cached markdown_it_py-4.0.0-py3-none-any.whl (87 kB)
Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [rich]2/3 [rich]own-it-py]
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install langgraph

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import json
from typing import TypedDict, List, Annotated, Optional

#Lanchain COmponents
from langchain_openai import ChatOpenAI
#from langchain_tavily import TavilySearchResults
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage, ToolMessage
from pydantic import BaseModel,Field

#Langgrahph components
from langgraph.graph import StateGraph, END
from langgraph.graph.message import AnyMessage,add_messages
#from langgraph.graph.prebuilt import ToolNode
from langgraph.prebuilt import ToolNode

#pretyy printing
from rich.console import Console
from rich.markdown import Markdown



os.environ['NEBIUS_API_KEY'] = ''
os.environ['LANGCHAIN_API_KEY'] = ''

# CRITICAL: Map NEBIUS_API_KEY to OPENAI_API_KEY for ChatNebius
# ChatNebius requires OPENAI_API_KEY, not NEBIUS_API_KEY
os.environ['OPENAI_API_KEY'] = ''
os.environ['TAVILY_API_KEY']=""



Creating and testing a web search tool

In [28]:
# Initialize the tool. We can set the max number of results to keep the context concise.
search_tool = TavilySearchResults(max_results=2)

# It's crucial to give the tool a clear name and description for the agent
search_tool.name = "web_search"
search_tool.description = "A tool that can be used to search the internet for up-to-date information on any topic, including news, events, and current affairs."

tools = [search_tool]
print(f"Tool '{search_tool.name}' created with description: '{search_tool.description}'")

console = Console()

# Let's test the tool directly to see its output format
print("\n--- Testing the tool directly ---")
test_query = "What was the score of the last Super Bowl?"
test_result = search_tool.invoke({"query": test_query})
console.print(f"[bold green]Query:[/bold green] {test_query}")
console.print("\n[bold green]Result:[/bold green]")
console.print(test_result)

Tool 'web_search' created with description: 'A tool that can be used to search the internet for up-to-date information on any topic, including news, events, and current affairs.'

--- Testing the tool directly ---


Query: What was the score of the last Super Bowl?

Result:

[
    {
        'title': 'Super Bowl Scores Last 10 Years | StatMuse',
        'url': 'https://www.statmuse.com/nfl/ask/super-bowl-scores-last-10-years',
        'content': 'Sign in/up\n\n NBA\n NFL\n CFB\n FC\n NHL\n MLB\n WNBA\n PGA\n Money\n\n Trending Sports\n 
Trending Money\n Trending Live\n\n Data & Glossary\n\nSign in/up\n\nSign in/up\n\n Home\n\n NBA\n NFL\n CFB\n FC\n 
NHL\n MLB\n WNBA\n PGA\n Money\n\n Scores\n\n Trending\n\n Trending Sports\n Trending Money\n Trending Live\n\n 
Examples\n\n Data & Glossary\n\n Gallery\n\n About\n\n Blog\n\n Shop\n\n# The Philadelphia Eagles beat the Kansas 
City Chiefs, 40 to 22, in Super Bowl LIX on February 9.\n\nSun, Feb 9, 2025\n\nSun, Feb 11, 
2024\n\n22\n\nChiefs\n\nFinal [...] Sun, Feb 12, 2023\n\nChiefs\n\nEagles\n\nFinal\n\nSun, Feb 13, 
2022\n\nBengals\n\n20\n\nFinal\n\nSun, Feb 7, 2021\n\nChiefs\n\n9\n\nBuccaneers\n\n31\n\nFinal\n\nSun, Feb 2, 
2020\n\n49ers\n\n20\n\nChiefs\n\n31\n\nFinal\n\nSun, Feb 3, 2019\n\nPatriots\n\n13\n\nRams\n\n3\n\nFinal\n\nSun, 
Feb 4, 2018\n\nEagles\n\n41\n\nPatriots\n\n33\n\nFinal\n\nSun, Feb 5, 
2017\n\nPatriots\n\n34\n\nFalcons\n\n28\n\nFinal\n\nSun, Feb 7, 
2016\n\nPanthers\n\n10\n\nBroncos\n\n24\n\nFinal\n\n### Related Searches',
        'score': 0.6315943
    },
    {
        'title': 'Super Bowl LVII - Wikipedia',
        'url': 'https://en.wikipedia.org/wiki/Super_Bowl_LVII',
        'content': "The Chiefs won the game 38–35 on a game-winning field goal by Harrison Butker. Butker's 
game-winning kick was set up by a pivotal defensive holding call on Philadelphia cornerback James Bradberry, which 
was criticized by some observers but supported by others, including Bradberry himself. The 73 combined points made 
this the third-highest scoring Super Bowl game, and the 35 points scored by the Eagles were the most by the losing 
team in the Super Bowl. Chiefs quarterback Patrick Mahomes was [...] Eagles retook the lead, 14–7, on a 45-yard 
touchdown pass from Hurts to wide receiver A. J. Brown. [...] This was the last Super Bowl that George Toma would 
serve as groundskeeper for, having previously served as groundskeeper for every Super Bowl since the event's 
creation. The field conditions were criticized by players, particularly for slipperiness.\n\nKansas City and 
Philadelphia would meet for a rematch two years later in Super Bowl LIX, in which the Eagles avenged their loss 
against the Chiefs, 40–22. The victory secured their second Super Bowl title, after Super Bowl LII seven years 
prior.",
        'score': 0.4902356
    }
]

Building the tool, Using AGnt with LangGraph

In [29]:
class AgentState(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]




In [30]:
llm=ChatOpenAI(model_name="gpt-4o-mini",temperature=0)

#Bind the tools to the LLM, amking it tool aware
llm_with_tools=llm.bind_tools(tools)

print("LLM has been bound with the provided tools and is now tool-aware.")

LLM has been bound with the provided tools and is now tool-aware.


Defining the Agent Nodes


In [31]:
def agent_node(state:AgentState):
    """The primary node that calls the llm to decide next action"""
    console.print("--AGENT: thinking")
    response=llm_with_tools.invoke(state["messages"])
    return {"messages":[response]}

#The Tool Node is a prebuilt node from LangGraph that executes tools
tool_node=ToolNode(tools)
print("Agent node and Tool node have been defined.")



Agent node and Tool node have been defined.


Defining the conditional ROuter

In [32]:
def router_function(state: AgentState)->str:
    """Inspects the agent's last messaeg to decide the next step"""
    last_message=state["messages"][-1]
    if last_message.tool_calls:
        #The agent has requested a tool call
        console.print("---Router: Decision is to call a tool. ---")
        return "call_tool"
    else:
        #The agent has provided a final answer
        console.print("__Router: Decision is to finish")
        return "__end__"
    
print("Router function defined")


Router function defined


Assembling and running workflow

In [37]:
graph_builder=StateGraph(AgentState)

#Add the nodes
graph_builder.add_node("agent",agent_node)
graph_builder.add_node("call_tool",tool_node)

#Set the entry point
graph_builder.set_entry_point('agent')

#Add the conditional router
graph_builder.add_conditional_edges("agent",router_function)

#Add the edge from the tool node back to the agent to complete loop
graph_builder.add_edge("call_tool","agent")

#Compile the graph
tool_agent_app=graph_builder.compile()
print("Tool -using agent graph compiled successfully")

# Visualize the graph
try:
    from IPython.display import Image, display
    png_image = tool_agent_app.get_graph().draw_png()
    display(Image(png_image))
except Exception as e:
    print(f"Graph visualization failed: {e}. Please ensure pygraphviz is installed.")

Tool -using agent graph compiled successfully
Graph visualization failed: Install pygraphviz to draw graphs: `pip install pygraphviz`.. Please ensure pygraphviz is installed.


End to ENd execution

In [42]:
user_query="What is the price of bitcoin and worldcoin today"
initial_input={
    "messages":[("user",user_query)]
    }

console.print(f"[bold cyan] Kcking off tool use workflow for request:[/bold cyan] '{user_query}'\n")

for chunk in tool_agent_app.stream(initial_input,stream_mode="values"):
    chunk["messages"][-1].pretty_print()
    console.print("\n--\n")

console.print("\n [bold green]✅ Tool Use workflow complete![/bold green]")

 Kcking off tool use workflow for request: 'What is the price of bitcoin and worldcoin today'

================================ Human Message =================================

What is the price of bitcoin and worldcoin today


--

--AGENT: thinking

---Router: Decision is to call a tool. ---

...

--AGENT: thinking

__Router: Decision is to finish

================================== Ai Message ==================================

As of today, the prices for Bitcoin and Worldcoin are as follows:

- **Bitcoin (BTC)**: $93,400.49 USD
  - 24-hour trading volume: $57,295,790,725 USD
  - Market cap: $1,864,254,154,003 USD
  - [More details on CoinMarketCap](https://coinmarketcap.com/currencies/bitcoin/)

- **Worldcoin (WLD)**: $0.629255 USD
  - 24-hour trading volume: $127,073,443 USD
  - Market cap: $1,516,074,503 USD
  - [More details on CoinMarketCap](https://coinmarketcap.com/currencies/worldcoin-org/)

Both cryptocurrencies have shown some movement in the last 24 hours, with Bitcoin up 3.83% and Worldcoin up 6.64%.


--

✅ Tool Use workflow complete!

Evaluating With llm as a judge

In [43]:
class ToolUseEvaluation(BaseModel):
    """Schema fro evaluating the agents tool use and fianl answer"""
    tool_selection_score: int = Field(description="Score 1-5 on whether the agent chose the correct tool for the task.")
    tool_input_score: int = Field(description="Score 1-5 on how well-formed and relevant the input to the tool was.")
    synthesis_quality_score: int = Field(description="Score 1-5 on how well the agent integrated the tool's output into its final answer.")
    justification: str = Field(description="A brief justification for the scores.")

judge_llm = llm.with_structured_output(ToolUseEvaluation)

# To evaluate, we need to reconstruct the full conversation trace
final_answer = tool_agent_app.invoke(initial_input)
conversation_trace = "\n".join([f"{m.type}: {m.content or ''} {getattr(m, 'tool_calls', '')}" for m in final_answer['messages']])

def evaluate_tool_use(trace: str):
    prompt = f"""You are an expert judge of AI agents. Evaluate the following conversation trace based on the agent's tool use on a scale of 1-5. Provide a brief justification.
    
    Conversation Trace:
    ```
    {trace}
    ```
    """
    return judge_llm.invoke(prompt)

console.print("--- Evaluating Tool Use Performance ---")
evaluation = evaluate_tool_use(conversation_trace)
console.print(evaluation.model_dump())


--AGENT: thinking

---Router: Decision is to call a tool. ---

--AGENT: thinking

__Router: Decision is to finish

--- Evaluating Tool Use Performance ---

{
    'tool_selection_score': 5,
    'tool_input_score': 5,
    'synthesis_quality_score': 5,
    'justification': "The agent effectively selected the appropriate tool (web search) to retrieve real-time 
cryptocurrency prices, which was the correct choice for the task. The inputs to the tool were well-formed and 
directly relevant to the user's query about Bitcoin and Worldcoin prices. The agent synthesized the information 
accurately, providing clear and concise answers along with relevant links for further details."
}